In [1]:
import tensorflow as tf

In [2]:
X = tf.range(10)

In [3]:
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [4]:
for i in dataset:
  print(i)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [5]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# 超参数
batch_size = 128
num_epochs = 10

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 将图像数据集转换成TFRecord格式
tfrecord_filename = 'mnist.tfrecords'
with tf.io.TFRecordWriter(tfrecord_filename) as writer:
    for i in range(x_train.shape[0]):
        example = tf.train.Example(features=tf.train.Features(feature={
            'image': tf.train.Feature(int64_list=tf.train.Int64List(value=x_train[i].reshape(-1).tolist())),
            'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[y_train[i]]))
        }))
        writer.write(example.SerializeToString())

# 定义解析函数
def parse_function(example_proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([784], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example_proto, feature_description)
    return tf.cast(example['image'], tf.float32) / 255., example['label']

# 读取并预处理数据
dataset = tf.data.TFRecordDataset(tfrecord_filename)
dataset = dataset.map(parse_function)
dataset = dataset.shuffle(buffer_size=10000)
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# 构建模型
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(784, )),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 训练模型
model.fit(dataset, epochs=num_epochs)

# 评估模型
test_loss, test_acc = model.evaluate(x_test.reshape((-1, 784))/255., y_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)


11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
469/469 [==============================] - 7s 12ms/step - loss: 0.3532 - accuracy: 0.9036
Epoch 2/10
469/469 [==============================] - 7s 13ms/step - loss: 0.1670 - accuracy: 0.9527
Epoch 3/10
469/469 [==============================] - 7s 14ms/step - loss: 0.1205 - accuracy: 0.9656
Epoch 4/10
469/469 [==============================] - 11s 21ms/step - loss: 0.0942 - accuracy: 0.9726
Epoch 5/10
469/469 [==============================] - 5s 10ms/step - loss: 0.0765 - accuracy: 0.9776
Epoch 6/10
469/469 [==============================] - 5s 11ms/step - loss: 0.0635 - accuracy: 0.9814
Epoch 7/10
469/469 [==============================] - 6s 11ms/step - loss: 0.0536 - accuracy: 0.9844
Epoch 8/10
469/469 [==============================] - 5s 10ms/step - loss: 0.0457 - accuracy: 0.9872
Epoch 9/10
469/469 [==============================] - 7s 13ms/step - loss: 0.0379 - accuracy: 0.9894
Epoch 10/10
313/313 [====